In [112]:
import pandas as pd

data = pd.read_csv("ETFs.csv")
data

,ticker,isin,fund_name,inception_date,category,rating,analyst_rating,risk_rating,performance_rating,investment_strategy,...,fund_return_2016_q4,fund_return_2016_q3,fund_return_2016_q2,fund_return_2016_q1,fund_return_2015_q4,fund_return_2015_q3,fund_return_2015_q2,fund_return_2015_q1,quarters_up,quarters_down
0,0P00000C5S,CH0008899764,iShares SMI® ETF (CH),1999-10-06,Switzerland Equity,3.0,Negative,3.0,3.0,The Fund aims to achieve a return on your inve...,...,1.45,5.10,10.47,-4.09,4.18,-4.00,-3.96,10.56,15,8
1,0P00000FYY,SE0000693293,XACT OMXS30 UCITS ETF,2000-10-30,Sweden Equity,2.0,NaN,2.0,2.0,The fund is an exchange-traded index fund and ...,...,5.21,11.32,1.66,1.43,4.99,-6.15,-7.21,10.11,16,7
2,0P00000GCE,DE000A0H08N1,iShares STOXX Europe 600 Personal & Household ...,2002-07-08,Sector Equity Consumer Goods & Services,3.0,NaN,3.0,3.0,iShares STOXX Europe 600 Personal & Household ...,...,-1.51,5.34,8.17,6.48,4.83,4.12,-2.03,9.24,17,6
3,0P00000GF2,DE0006289465,iShares eb.rexx® Government Germany UCITS ETF ...,2003-02-04,EUR Government Bond,2.0,NaN,2.0,2.0,iShares eb.rexx Government Germany (DE) is an ...,...,-2.14,4.21,6.04,9.67,0.05,4.96,-3.72,-5.67,13,10
4,0P00000GKU,IE0032523478,iShares € Corp Bond Large Cap UCITS ETF EUR (D...,2003-03-17,EUR Corporate Bond,3.0,NaN,3.0,3.0,The investment objective of this Fund is to pr...,...,-2.74,5.58,6.26,10.55,1.38,3.78,-5.02,-5.84,16,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9490,0P0001K8E2,LU2099288503,Lyxor Core Global Government Bond (DR) UCITS E...,2020-06-24,Global Bond,NaN,NaN,NaN,NaN,The investment objective of MULTI UNITS LUXEMB...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
9491,0P0001K9DU,LU1220245556,Lyxor MSCI Pacific Ex Japan UCITS ETF - D-USD,2015-04-29,Pacific ex-Japan Equity,3.0,NaN,4.0,3.0,The Lyxor MSCI Pacific Ex Japan UCITS ETF is a...,...,2.25,11.22,8.06,4.26,11.13,-12.89,NaN,NaN,13,8
9492,0P0001K9DV,LU1923627332,Lyxor MSCI Russia UCITS ETF Dist,2019-03-14,Russia Equity,2.0,Negative,4.0,2.0,The investment objective of the MULTI UNITS LU...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2
9493,0P0001K9DW,LU1650492256,MULTI-UNITS LUXEMBOURG - Lyxor FTSE 100 UCITS ...,2017-11-09,UK Large-Cap Equity,NaN,Bronze,NaN,NaN,The Lyxor FTSE 100 UCITS ETF is a UCITS compli...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,5


In [113]:
columns = ["fund_name", 
"sector_basic_materials", 
"sector_consumer_cyclical", "sector_financial_services", "sector_real_estate", "sector_consumer_defensive", "sector_healthcare", "sector_utilities", "sector_communication_services", "sector_energy", "sector_industrials", "sector_technology",
"management_fees", "sustainability_rank",
"fund_return_2019", "fund_return_2018", "fund_return_2017", "fund_return_2016", "fund_return_2015" , "dividend_yield_factor" , "asset_stock" , "asset_bond" , "asset_cash" , "fund_size"]
df = data[columns]
df

,fund_name,sector_basic_materials,sector_consumer_cyclical,sector_financial_services,sector_real_estate,sector_consumer_defensive,sector_healthcare,sector_utilities,sector_communication_services,sector_energy,...,fund_return_2019,fund_return_2018,fund_return_2017,fund_return_2016,fund_return_2015,dividend_yield_factor,asset_stock,asset_bond,asset_cash,fund_size
0,iShares SMI® ETF (CH),8.72,4.57,17.72,NaN,17.24,41.93,NaN,1.24,NaN,...,26.84,-2.77,11.71,12.96,6.20,3.23,99.81,0.00,0.19,1.976640e+09
1,XACT OMXS30 UCITS ETF,3.72,6.06,22.15,NaN,6.71,4.14,NaN,5.15,NaN,...,19.17,-8.91,8.41,20.77,0.68,1.45,99.99,0.00,0.01,7.934210e+09
2,iShares STOXX Europe 600 Personal & Household ...,NaN,42.49,NaN,NaN,54.01,NaN,NaN,1.80,NaN,...,26.84,-14.82,16.63,19.50,16.82,2.59,99.46,0.00,0.54,1.190300e+08
3,iShares eb.rexx® Government Germany UCITS ETF ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.79,2.11,2.76,18.60,-4.63,NaN,0.00,99.86,0.14,3.593900e+08
4,iShares € Corp Bond Large Cap UCITS ETF EUR (D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.24,-0.27,5.63,20.63,-5.91,NaN,0.00,96.04,-0.03,3.903070e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9490,Lyxor Core Global Government Bond (DR) UCITS E...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,100.00,0.00,1.948000e+07
9491,Lyxor MSCI Pacific Ex Japan UCITS ETF - D-USD,11.29,5.14,36.19,11.32,5.64,9.95,4.39,3.32,2.13,...,13.74,-4.66,14.87,28.11,NaN,4.67,100.00,0.00,0.00,1.506000e+07
9492,Lyxor MSCI Russia UCITS ETF Dist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,2.153500e+08
9493,MULTI-UNITS LUXEMBOURG - Lyxor FTSE 100 UCITS ...,12.08,8.71,16.21,1.25,19.46,12.93,4.33,6.71,8.35,...,17.05,-8.87,NaN,NaN,NaN,4.04,99.16,0.00,0.04,3.012300e+08


In [114]:
df.isna().sum()

fund_name                           0
sector_basic_materials           4576
sector_consumer_cyclical         4385
sector_financial_services        4510
sector_real_estate               5081
sector_consumer_defensive        4685
sector_healthcare                4838
sector_utilities                 5008
sector_communication_services    4490
sector_energy                    5029
sector_industrials               4403
sector_technology                4705
management_fees                   409
sustainability_rank              3096
fund_return_2019                 1834
fund_return_2018                 3161
fund_return_2017                 3865
fund_return_2016                 4606
fund_return_2015                 5202
dividend_yield_factor            3502
asset_stock                         0
asset_bond                          0
asset_cash                          0
fund_size                          14
dtype: int64

first delete management_fees and make sus_rank to 0 to expect the worst.
also all sectors to 0 as they dont have any representation

In [115]:
df = df.dropna(subset=["fund_size"])
df = df.dropna(subset=["management_fees"])
df["management_fees"] = df["management_fees"] / 100
df["dividend_yield_factor"] = df["dividend_yield_factor"] / 100

asset_columns = df.filter(regex="asset_*").columns
df[asset_columns] = df[asset_columns] / 100

df["dividend_yield_factor"] = df["dividend_yield_factor"].fillna(0)
df["sustainability_rank"] = df["sustainability_rank"].fillna(0)

sector_columns = df.filter(regex="sector_*").columns
df[sector_columns] = df[sector_columns].fillna(0)

print(df.isna().sum())
df.shape

fund_name                           0
sector_basic_materials              0
sector_consumer_cyclical            0
sector_financial_services           0
sector_real_estate                  0
sector_consumer_defensive           0
sector_healthcare                   0
sector_utilities                    0
sector_communication_services       0
sector_energy                       0
sector_industrials                  0
sector_technology                   0
management_fees                     0
sustainability_rank                 0
fund_return_2019                 1640
fund_return_2018                 2875
fund_return_2017                 3569
fund_return_2016                 4292
fund_return_2015                 4876
dividend_yield_factor               0
asset_stock                         0
asset_bond                          0
asset_cash                          0
fund_size                           0
dtype: int64


(9081, 24)

if a fund is new and has no return we set it to 0 to make preference for older funds with more trustworthy record, but we can tune that later.

In [116]:
fund_columns = df.filter(regex="fund_return").columns
df[fund_columns] = df[fund_columns].fillna(0) / 100

df["fund_return"] = df.filter(regex="fund_return*").mean(axis=1)
df["fund_risk"] = df.filter(regex="fund_return*").std(axis=1)
print(df.isna().sum())


fund_name                        0
sector_basic_materials           0
sector_consumer_cyclical         0
sector_financial_services        0
sector_real_estate               0
sector_consumer_defensive        0
sector_healthcare                0
sector_utilities                 0
sector_communication_services    0
sector_energy                    0
sector_industrials               0
sector_technology                0
management_fees                  0
sustainability_rank              0
fund_return_2019                 0
fund_return_2018                 0
fund_return_2017                 0
fund_return_2016                 0
fund_return_2015                 0
dividend_yield_factor            0
asset_stock                      0
asset_bond                       0
asset_cash                       0
fund_size                        0
fund_return                      0
fund_risk                        0
dtype: int64


In [117]:
test = df.loc[1:10]
test

,fund_name,sector_basic_materials,sector_consumer_cyclical,sector_financial_services,sector_real_estate,sector_consumer_defensive,sector_healthcare,sector_utilities,sector_communication_services,sector_energy,...,fund_return_2017,fund_return_2016,fund_return_2015,dividend_yield_factor,asset_stock,asset_bond,asset_cash,fund_size,fund_return,fund_risk
1,XACT OMXS30 UCITS ETF,3.72,6.06,22.15,0.00,6.71,4.14,0.00,5.15,0.00,...,0.0841,0.2077,0.0068,0.0145,0.9999,0.0000,0.0001,7.934210e+09,0.08024,0.112030
2,iShares STOXX Europe 600 Personal & Household ...,0.00,42.49,0.00,0.00,54.01,0.00,0.00,1.80,0.00,...,0.1663,0.1950,0.1682,0.0259,0.9946,0.0000,0.0054,1.190300e+08,0.12994,0.143911
3,iShares eb.rexx® Government Germany UCITS ETF ...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0276,0.1860,-0.0463,0.0000,0.0000,0.9986,0.0014,3.593900e+08,0.02810,0.085197
4,iShares € Corp Bond Large Cap UCITS ETF EUR (D...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0563,0.2063,-0.0591,0.0000,0.0000,0.9604,-0.0003,3.903070e+09,0.04064,0.090529
5,iShares $ Corp Bond UCITS ETF USD (Dist),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-0.0244,0.2728,0.0484,0.0000,0.0000,0.9647,0.0042,1.009780e+10,0.09006,0.104841
6,UBS ETF - MSCI EMU UCITS ETF (EUR) A-dis,7.06,14.79,13.76,1.92,10.14,9.36,7.16,6.25,3.67,...,0.1761,0.2039,0.0546,0.0260,0.9930,0.0000,0.0070,3.890450e+09,0.10320,0.124321
7,UBS ETF - MSCI Japan UCITS ETF (JPY) A-dis,5.19,15.09,9.06,3.55,7.33,11.43,1.40,9.69,1.28,...,0.1366,0.2092,0.1686,0.0275,0.9963,0.0000,0.0037,2.707825e+11,0.11498,0.103185
8,Lyxor DJ Global Titans 50 UCITS ETF Dist,0.78,13.45,8.92,0.00,10.50,13.84,0.00,14.28,2.59,...,0.1088,0.3004,0.0671,0.0188,1.0000,0.0000,0.0000,9.666000e+07,0.14956,0.111378
9,Lyxor Dow Jones Industrial Average UCITS ETF Dist,1.13,14.89,16.90,0.00,8.04,19.08,0.00,4.43,1.73,...,0.1592,0.3760,0.0514,0.0246,1.0000,0.0000,0.0000,2.003100e+08,0.15924,0.127504
10,iShares Core FTSE 100 UCITS ETF GBP (Dist),11.81,8.75,16.88,1.27,19.01,12.71,4.01,6.87,8.89,...,0.1313,0.1867,-0.0171,0.0408,0.9820,0.0000,0.0101,8.125630e+09,0.07644,0.109964


In [118]:

corr_matrix = df[sector_columns].T.corr()
corr_matrix = corr_matrix.fillna(0)
print(corr_matrix)

          0         1         2     3     4     5         6         7     \
0     1.000000  0.108965  0.089980   0.0   0.0   0.0  0.299356  0.206325   
1     0.108965  1.000000 -0.105342   0.0   0.0   0.0  0.730856  0.771375   
2     0.089980 -0.105342  1.000000   0.0   0.0   0.0  0.353204  0.144005   
3     0.000000  0.000000  0.000000   0.0   0.0   0.0  0.000000  0.000000   
4     0.000000  0.000000  0.000000   0.0   0.0   0.0  0.000000  0.000000   
...        ...       ...       ...   ...   ...   ...       ...       ...   
9490  0.000000  0.000000  0.000000   0.0   0.0   0.0  0.000000  0.000000   
9491  0.363769  0.374686 -0.197563   0.0   0.0   0.0  0.304021  0.006478   
9492  0.000000  0.000000  0.000000   0.0   0.0   0.0  0.000000  0.000000   
9493  0.654573  0.175379  0.467633   0.0   0.0   0.0  0.382170  0.035228   
9494  0.000000  0.000000  0.000000   0.0   0.0   0.0  0.000000  0.000000   

          8         9     ...      9478      9479      9480  9488  9489  9490  \
0     

In [119]:
# corr_matrix_pyomo = {}

# Iterate over the rows and columns of the correlation matrix
# for fund_i_index, fund_i in enumerate(corr_matrix.index):
#     print(fund_i_index)
#     for fund_j_index, fund_j in enumerate(corr_matrix.index):
#         print(fund_j)
#         print(corr_matrix)
#         corr_matrix_pyomo[(fund_i_index, fund_j_index)] = corr_matrix[fund_i_index, fund_j_index]
#         
# corr_matrix_pyomo

In [130]:

from pyomo.environ import *

model = ConcreteModel()
# Definition of data (sets and params)

model.Funds = Set(initialize = test.index, doc = "Funds")

model.Return = Param(model.Funds, initialize = test["fund_return"].to_dict(), doc="Return")
model.Risk = Param(model.Funds, initialize = test["fund_risk"].to_dict(), doc="Risk")
#model.Correlation = Param(model.Funds, model.Funds, initialize = corr_matrix, doc = "Correlation")
model.SustainabilityRank = Param(model.Funds, initialize = test["sustainability_rank"].to_dict(), doc="Sustainability Rank")
model.MgnFee = Param(model.Funds, initialize = test["management_fees"].to_dict(), doc="Management Fees")
model.DivYield = Param(model.Funds, initialize = test["dividend_yield_factor"].to_dict(), doc="Dvidend Yield")
model.AssetStockPercentage = Param(model.Funds, initialize = test["asset_stock"].to_dict(), doc="Asset Stock Percentage")
model.AssetBondPercentage = Param(model.Funds, initialize = test["asset_bond"].to_dict(), doc="Asset Bond Percentage")
model.AssetCashPercentage = Param(model.Funds, initialize = test["asset_cash"].to_dict(), doc="Asset Cash Percentage")
model.FundSize = Param(model.Funds, initialize = test["fund_size"].to_dict(), doc="Fund Size")

model.x = Var(model.Funds, within=NonNegativeReals)

def profit_rule(model):
    return sum((model.Return[i] - model.MgnFee[i])*model.x[i] for i in model.Funds)
model.obj = Objective(rule=profit_rule, sense = maximize, doc = "Total revenue")

def budget_constraint(model):
    return (sum(model.x[i] for i in model.Funds) <= 1)
model.budget = Constraint(rule=budget_constraint,doc="Maximum allocation is 100%")

MAX_INVESTMENT_RISK = 0.2
def max_risk_constraint(model):
    return sum(model.Risk[i] * model.x[i] for i in model.Funds) <= MAX_INVESTMENT_RISK
model.max_risk = Constraint(rule=max_risk_constraint,doc="Investment risk")

MAX_ALLOCATION = 0.3
def max_allocation_constraint(model, fund):
    return (model.x[fund] <= MAX_ALLOCATION)
model.max_allocation = Constraint(model.Funds, rule=max_allocation_constraint, doc="Max allocation for a fund")

MIN_SUSTAINABILITY_RANK = 2
def min_sustainability_rank_constraint(model):
    return (sum(model.SustainabilityRank[i] * model.x[i] for i in model.Funds) >= MIN_SUSTAINABILITY_RANK)
model.min_sustainability_rank = Constraint(rule=min_sustainability_rank_constraint,doc="Min sustainability rank")

MIN_FUND_SIZE = 0
def min_fund_size_constraint(model):
    return (sum(model.FundSize[i] * model.x[i] for i in model.Funds) >= MIN_FUND_SIZE)
model.min_fund_size = Constraint(rule=min_fund_size_constraint,doc="Min fund size")

MAX_FUND_SIZE = 1e10
def max_fund_size_constraint(model):
    return (sum(model.FundSize[i] * model.x[i] for i in model.Funds) <= MAX_FUND_SIZE)
model.max_fund_size = Constraint(rule=max_fund_size_constraint,doc="Max fund size")

MIN_DIVIDEND_YIELD_FACTOR = 0.01
def min_dividend_yield_factor_constraint(model):
    return (sum(model.DivYield[i] * model.x[i] for i in model.Funds) >= MIN_DIVIDEND_YIELD_FACTOR)
model.min_divided_yield = Constraint(rule=min_dividend_yield_factor_constraint,doc="Min dividend yield")

MAX_DIVIDEND_YIELD_FACTOR = 1
def max_dividend_yield_factor_constraint(model):
    return (sum(model.DivYield[i] * model.x[i] for i in model.Funds) <= MAX_DIVIDEND_YIELD_FACTOR)
model.max_divided_yield = Constraint(rule=max_dividend_yield_factor_constraint,doc="Max dividend yield")

MIN_ASSET_STOCK = 0
def min_asset_stock_constraint(model):
    return (sum(model.AssetStockPercentage[i] * model.x[i] for i in model.Funds) >= MIN_ASSET_STOCK)
model.min_asset_stock = Constraint(rule=min_asset_stock_constraint,doc="Min asset stock")

MAX_ASSET_STOCK = 1
def max_asset_stock_constraint(model):
    return (sum(model.AssetStockPercentage[i] * model.x[i] for i in model.Funds) <= MAX_ASSET_STOCK)
model.max_asset_stock = Constraint(rule=max_asset_stock_constraint,doc="Max asset stock")

MIN_ASSET_BOND = 0
def min_asset_bond_constraint(model):
    return (sum(model.AssetBondPercentage[i] * model.x[i] for i in model.Funds) >= MIN_ASSET_BOND)
model.min_asset_bond = Constraint(rule=min_asset_bond_constraint,doc="Min asset bond")

MAX_ASSET_BOND = 1
def max_asset_bond_constraint(model):
    return (sum(model.AssetBondPercentage[i] * model.x[i] for i in model.Funds) <= MAX_ASSET_BOND)
model.max_asset_bond = Constraint(rule=max_asset_bond_constraint,doc="Max asset bond")

MIN_ASSET_CASH = 0.2
def min_asset_cash_constraint(model):
    return (sum(model.AssetBondPercentage[i] * model.x[i] for i in model.Funds) >= MIN_ASSET_CASH)
model.min_asset_cash = Constraint(rule=min_asset_cash_constraint,doc="Min asset cash")

MAX_ASSET_CASH = 1
def max_asset_cash_constraint(model):
    return (sum(model.AssetBondPercentage[i] * model.x[i] for i in model.Funds) <= MAX_ASSET_CASH)
model.max_asset_cash = Constraint(rule=max_asset_cash_constraint,doc="Max asset cash")

Solver = SolverFactory('glpk')

# Obtain the solution
Results = Solver.solve(model)

# Display the solution
model.display()


Model unknown

  Variables:
    x : Size=10, Index=Funds
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          1 :     0 :               0.0 :  None : False : False : NonNegativeReals
          2 :     0 : 0.192681662693065 :  None : False : False : NonNegativeReals
          3 :     0 :               0.0 :  None : False : False : NonNegativeReals
          4 :     0 :               0.0 :  None : False : False : NonNegativeReals
          5 :     0 : 0.207318337306935 :  None : False : False : NonNegativeReals
          6 :     0 :               0.0 :  None : False : False : NonNegativeReals
          7 :     0 :               0.0 :  None : False : False : NonNegativeReals
          8 :     0 :               0.3 :  None : False : False : NonNegativeReals
          9 :     0 :               0.3 :  None : False : False : NonNegativeReals
         10 :     0 :               0.0 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=N

In [ ]:
model.obj()
test

,fund_name,sector_basic_materials,sector_consumer_cyclical,sector_financial_services,sector_real_estate,sector_consumer_defensive,sector_healthcare,sector_utilities,sector_communication_services,sector_energy,...,fund_return_2017,fund_return_2016,fund_return_2015,dividend_yield_factor,asset_stock,asset_bond,asset_cash,fund_size,fund_return,fund_risk
1,XACT OMXS30 UCITS ETF,3.72,6.06,22.15,0.00,6.71,4.14,0.00,5.15,0.00,...,0.0841,0.2077,0.0068,1.45,0.9999,0.0000,0.0001,7.934210e+09,0.08024,0.112030
2,iShares STOXX Europe 600 Personal & Household ...,0.00,42.49,0.00,0.00,54.01,0.00,0.00,1.80,0.00,...,0.1663,0.1950,0.1682,2.59,0.9946,0.0000,0.0054,1.190300e+08,0.12994,0.143911
3,iShares eb.rexx® Government Germany UCITS ETF ...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0276,0.1860,-0.0463,0.00,0.0000,0.9986,0.0014,3.593900e+08,0.02810,0.085197
4,iShares € Corp Bond Large Cap UCITS ETF EUR (D...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0563,0.2063,-0.0591,0.00,0.0000,0.9604,-0.0003,3.903070e+09,0.04064,0.090529
5,iShares $ Corp Bond UCITS ETF USD (Dist),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-0.0244,0.2728,0.0484,0.00,0.0000,0.9647,0.0042,1.009780e+10,0.09006,0.104841
6,UBS ETF - MSCI EMU UCITS ETF (EUR) A-dis,7.06,14.79,13.76,1.92,10.14,9.36,7.16,6.25,3.67,...,0.1761,0.2039,0.0546,2.60,0.9930,0.0000,0.0070,3.890450e+09,0.10320,0.124321
7,UBS ETF - MSCI Japan UCITS ETF (JPY) A-dis,5.19,15.09,9.06,3.55,7.33,11.43,1.40,9.69,1.28,...,0.1366,0.2092,0.1686,2.75,0.9963,0.0000,0.0037,2.707825e+11,0.11498,0.103185
8,Lyxor DJ Global Titans 50 UCITS ETF Dist,0.78,13.45,8.92,0.00,10.50,13.84,0.00,14.28,2.59,...,0.1088,0.3004,0.0671,1.88,1.0000,0.0000,0.0000,9.666000e+07,0.14956,0.111378
9,Lyxor Dow Jones Industrial Average UCITS ETF Dist,1.13,14.89,16.90,0.00,8.04,19.08,0.00,4.43,1.73,...,0.1592,0.3760,0.0514,2.46,1.0000,0.0000,0.0000,2.003100e+08,0.15924,0.127504
10,iShares Core FTSE 100 UCITS ETF GBP (Dist),11.81,8.75,16.88,1.27,19.01,12.71,4.01,6.87,8.89,...,0.1313,0.1867,-0.0171,4.08,0.9820,0.0000,0.0101,8.125630e+09,0.07644,0.109964


In [ ]:
model.pprint()

1 Set Declarations
    Funds : Funds
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

6 Param Declarations
    AssetBond : Dvidend Yield
        Size=10, Index=Funds, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :  1.45
          2 :  2.59
          3 :   0.0
          4 :   0.0
          5 :   0.0
          6 :   2.6
          7 :  2.75
          8 :  1.88
          9 :  2.46
         10 :  4.08
    DivYield : Dvidend Yield
        Size=10, Index=Funds, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :  1.45
          2 :  2.59
          3 :   0.0
          4 :   0.0
          5 :   0.0
          6 :   2.6
          7 :  2.75
          8 :  1.88
          9 :  2.46
         10 :  4.08
    MgnFee : Management Fees
        Size=10, Index=Funds, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :  0.00